# Performance of the crypto_metamath_model

### Set up the GPU

In [1]:
import os
gpu = os.environ["CUDA_VISIBLE_DEVICES"]="3"

### Some Conclusions:

1. 

In [2]:
import gc

#Force garbage collection
gc.collect()

31

### Import libraries

In [3]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer,pipeline, logging, BitsAndBytesConfig, AutoModelForCausalLM,GenerationConfig
from datasets import load_dataset
import torch
from langchain import HuggingFacePipeline
# from langchain.agents.agent_toolkits import create_python_agent
# from langchain.tools.python.tool import PythonREPLTool
from langchain.chains import LLMMathChain, LLMChain
from langchain.agents import Tool, initialize_agent
from langchain.prompts import PromptTemplate
from langchain.agents.agent_types import AgentType

/projects/barman/crypto-llm-v2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-14 10:36:35.836126: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-14 10:36:35.886405: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-14 10:36:43.303439: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not

### Get the crypto model and the base model

In [4]:
# Get the finetuned model
output_dir = "../../results/metamath-crypto-14-05-00-30"

# base model
base = "meta-math/MetaMath-Mistral-7B"

### Get some testing prompts

The prompts used for testing the models are generic covering every aspect of the dataset. Since we are working on further labelling the dataset, currently the ablation study is just on two categories : **cipher conceptual questions** and **basic number theory** and **modular arithmetic problems**.

In [5]:
# testing prompts
prompts = {"conceptual" : ["What is symmetric encryption. Explain briefly",
                           "What are hashing functions? What are hash collisions? Answer briefly",
                           "What is a crystal cipher? Describe its strcuture. Answer breifly",
                           "What is Caeser cipher? Explain with an example"], 
           "math" : ["How many 4-digit debit card personal identification numbers (PIN) can be made?",
                    "How many ways can you arrange 3 people standing in line? Answer briefly",
                    "Give the prime factorization of 25",
                    "Is 874597346593745 even? Justify your answer",
                     "What is the lcm of 1 and 2",
                    "Is 55 divisible by 5?"]}

### Original Crypto-dataset - without augmented samples

In [6]:
# Load the crypto dataset
crypto_dataset = load_dataset("Manpa/cryptoqa-v1", split="test")

In [7]:
crypto_dataset[3]

{'question': 'Write one disadvantage of assymmetric encryption.',
 'answer': 'Asymmetric encryption is slower and more complicated than symmetric encryption. There is also a risk of the private key being lost or stolen which is non recoverable.',
 'type': 'orig',
 'category': 'word',
 'topic': 'cipher',
 'source': 'self'}

### Set the quantization parameters and the tokenizer

In [8]:
# Set the bits and bytes configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(output_dir)

## Test on the mathematical capabilities

In [9]:
DEFAULT_SYSTEM_PROMPT = """ You are a fine-tuned AI model who is a math genious. 
You can solve simple to moderate level mathematics problems. 
Follow a chain of thought approach while answering. Answer in brief. """

### Test on the base model on which cryptollama7b is trained on.

In [10]:
# Load the pretrained model
base_model = AutoModelForCausalLM.from_pretrained(
    base,
    load_in_4bit=True,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
    trust_remote_code=True,
    cache_dir="/projects/barman/cache"
)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.04s/it]


In [12]:
def generate_response(prompt,test_model):
    # Ignore warnings
    logging.set_verbosity(logging.CRITICAL)

    # Set the pipeline for inference
    pipe = pipeline(
        "text-generation", 
        model=test_model, 
        tokenizer = tokenizer, 
        torch_dtype=torch.bfloat16, 
        device_map= {"": 0}
    )
    # Print the results
    result = pipe(f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n ### Instruction:\n{prompt}\n\n### Response: Let's think step by step.\n",do_sample=True,
        max_new_tokens=512, 
        temperature=0.9, 
        top_k=50, 
        top_p=0.95,
        num_return_sequences=1)
    return result

In [14]:
result = generate_response(crypto_dataset[24]['question'],base_model)
print(f"Input:\n{crypto_dataset[24]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")

Input:
What is the sum of the first 10 natural numbers?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What is the sum of the first 10 natural numbers?

### Response: Let's think step by step.
The first natural number is 1.
The second natural number is 2.
The third natural number is 3.
And so on...
The tenth natural number is 10.
To find the sum of the first 10 natural numbers, we can add them up:
$1 + 2 + 3 + \dots + 10 = \boxed{55}$.



In [15]:
result = generate_response(crypto_dataset[26]['question'],base_model)
print(f"Input:\n{crypto_dataset[26]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[26]['answer']}")

Input:
Determine if the expression \(3 \cdot 3262 + 2\) is divisible by 3.

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
Determine if the expression \(3 \cdot 3262 + 2\) is divisible by 3.

### Response: Let's think step by step.
We know that a number is divisible by 3 if the sum of its digits is divisible by 3.
The sum of the digits of 3262 is $3 + 2 + 6 + 2 = 13$.
Since 13 is not divisible by 3, the expression \(3 \cdot 3262 + 2\) is not divisible by 3.

Ground Truth 
 The expression \(3 \cdot 3262 + 2\) is not divisible by 3 as 3 \cdot 3262 = 9786 is divisible by 3 but 2 is not divisible by 3.


In [16]:
result = generate_response(crypto_dataset[30]['question'],base_model)
print(f"Input:\n{crypto_dataset[30]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[30]['answer']}")

Input:
Determine if the expression \(3 \cdot 3262 + 2\) is divisible by 3.

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
Determine if the expression \(3 \cdot 3262 + 2\) is divisible by 3.

### Response: Let's think step by step.
We can rewrite the expression as $3 \cdot 3262 + 2 = 3 \cdot (1000 + 262 + 2) + 2$.
Simplifying, we get $3 \cdot 1264 + 2$.
Since $1264$ is divisible by $3$, we can conclude that the entire expression is divisible by $3$.
Therefore, the expression \(3 \cdot 3262 + 2\) is divisible by \(3\).
Ground Truth 
 The expression \(3 \cdot 3262 + 2\) is not divisible by 3 as 3 \cdot 3262 = 9786 is divisible by 3 but 2 is not divisible by 3.


In [17]:
result = generate_response(crypto_dataset[32]['question'],base_model)
print(f"Input:\n{crypto_dataset[32]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[32]['answer']}")

Input:
Is 3493 a prime ?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
Is 3493 a prime ?

### Response: Let's think step by step.
To determine if a number is prime, we need to check if it is divisible by any number other than 1 and itself.
We can start by checking if 3493 is divisible by 2, 3, 5, and 7, as these are the smallest prime numbers.
3493 is not divisible by 2, 3, 5, or 7.
Next, we can check if 3493 is divisible by any number between 2 and $\sqrt{3493}$.
We can see that $\sqrt{3493}$ is approximately 59.48.
So, we only need to check if 3493 is divisible by numbers between 2 and 59.
We find that 3493 is not divisible by any of these numbers.
Therefore, we can conclude that 3493 is a prime number.

Ground Truth 
 No, 3493 is not a prime number as 3493 can be divided by 1, 7, 499, and 3493.


In [18]:
result = generate_response(crypto_dataset[42]['question'],base_model)
print(f"Input:\n{crypto_dataset[42]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[42]['answer']}")

Input:
There are 3 juniors, 4 seniors, and 2 sophomores in a class. If a committee of 4 students is to be formed, what is the probability that the committee will have 2 juniors, 1 senior, and 1 sophomore?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
There are 3 juniors, 4 seniors, and 2 sophomores in a class. If a committee of 4 students is to be formed, what is the probability that the committee will have 2 juniors, 1 senior, and 1 sophomore?

### Response: Let's think step by step.
First, we need to choose 2 juniors from the 3 available. This can be done in $\binom{3}{2}=3$ ways.
Next, we need to choose 1 senior from the 4 available. This can be done in $\binom{4}{1}=4$ ways.
Finally, we need to choose 1 sophomore from the 2 available. This can be done in $\binom{2}{1}=2$ ways.
Therefore, the total number of ways to form a committee with 2 juniors, 1 senior, and 1 sophomore is $3 \cd

In [19]:
result = generate_response(crypto_dataset[39]['question'],base_model)
print(f"Input:\n{crypto_dataset[39]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[39]['answer']}")

Input:
A die is rolled twice. Let A be the event rolling a 6 on the first roll and let B be the event rolling a 3 on the second roll. Are A and B independent events?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
A die is rolled twice. Let A be the event rolling a 6 on the first roll and let B be the event rolling a 3 on the second roll. Are A and B independent events?

### Response: Let's think step by step.
The probability of rolling a 6 on the first roll is $\frac{1}{6}$.
The probability of rolling a 3 on the second roll is $\frac{1}{6}$.
Since the first roll does not affect the second roll, the events A and B are independent.
Therefore, the answer is $\boxed{\text{Yes}}$.
Ground Truth 
 Yes, A and B are independent events as the outcome of the first roll does not affect the outcome of the second roll.


### Test on the fine tuned METAMATH model

In [20]:
# Load the finetuned model from output_dir
ft_model =AutoPeftModelForCausalLM.from_pretrained(
    output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
    cache_dir="/projects/barman/cache",
)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.01s/it]


In [21]:
result = generate_response(crypto_dataset[24]['question'],ft_model)
print(f"Input:\n{crypto_dataset[24]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[24]['answer']}")

/projects/barman/crypto-llm-v2/lib/python3.12/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


Input:
What is the sum of the first 10 natural numbers?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What is the sum of the first 10 natural numbers?

### Response: Let's think step by step.
The sum of the first 10 natural numbers is defined as: $1+2+3+...+10=55$. 

There is no step-by-step guide to think in this scenario, so the answer is left to the imagination. 

Ground Truth 
 The sum of the first 10 natural numbers is 1+2+3+4+5+6+7+8+9+10 = 55. It can be also computed as n(n+1)/2 where n is the number of natural numbers.


In [22]:
result = generate_response(crypto_dataset[26]['question'],ft_model)
print(f"Input:\n{crypto_dataset[26]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[26]['answer']}")

Input:
Determine if the expression \(3 \cdot 3262 + 2\) is divisible by 3.

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
Determine if the expression \(3 \cdot 3262 + 2\) is divisible by 3.

### Response: Let's think step by step.
We can determine the divisibility of the expression \(3 \cdot 3262 + 2\) by considering the remainders of the digits being multiplied and the divisibility rule for 3. According to the divisibility rule for 3, a number is divisible by 3 if the sum of its digits is divisible by 3. 

Applying this rule, we evaluate \(3 \cdot 3262 + 2\). The sum of the digits of 3262 is 10. Furthermore, 10 divided by 3 leaves a remainder of 1. 

Therefore, the expression \(3 \cdot 3262 + 2\) is not divisible by 3. 

Alternatively, one can evaluate the remainder of 1 when dividing by 3. Let's check the divisibility with this method.

Applying this rule, we evaluate \(3 \cdot 3262 + 

In [23]:
result = generate_response(crypto_dataset[30]['question'],ft_model)
print(f"Input:\n{crypto_dataset[30]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[30]['answer']}")

Input:
Is 0.333333..... a rational number?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
Is 0.333333..... a rational number?

### Response: Let's think step by step.
Yes, 0.333333..... is a rational number. 
To confirm this, we can observe that 0.333333..... can be written as 3/9.
0.333333... can be expressed as 3/9, which confirms that it is a rational number. 
#### 9/12 

Ground Truth 
 Yes, 0.333333..... is a rational number as it can be expressed as 1/3.


In [26]:
result = generate_response(crypto_dataset[32]['question'],ft_model)
print(f"Input:\n{crypto_dataset[32]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[32]['answer']}")

Input:
Is 3493 a prime ?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
Is 3493 a prime ?

### Response: Let's think step by step.
No, 3493 is not a prime. It can be factored into 3 and 1127, both of which are prime numbers. 
3493 = 3 * 1127 
Hence, 3493 is not a prime number. 

Ground Truth 
 No, 3493 is not a prime number as 3493 can be divided by 1, 7, 499, and 3493.


In [27]:
result = generate_response(crypto_dataset[41]['question'],ft_model)
print(f"Input:\n{crypto_dataset[41]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[41]['answer']}")

Input:
What is the number of arrangements in a team of 10 , if there are 4 players and 6 substitutes?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What is the number of arrangements in a team of 10 , if there are 4 players and 6 substitutes?

### Response: Let's think step by step.
There are 10 people in the team, out of which 4 are players and the remaining 6 are substitutes. The number of ways in which these 10 people can be arranged in a sequence is 10!. However, this counts arrangements where players and substitutes are intermingled. 
To account for this, we need to divide 10! by the number of ways the players and substitutes can be arranged within their own group, i.e. 4! and 6! respectively. Thus, the final calculation becomes 10! / (4! * 6!). 
Evaluating this expression, we find that the number of arrangements in a team of 10, with 4 players and 6 substitutes, is 40320. 
So, the

In [29]:
result = generate_response(crypto_dataset[39]['question'],ft_model)
print(f"Input:\n{crypto_dataset[3]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[39]['answer']}")

Input:
A die is rolled twice. Let A be the event rolling a 6 on the first roll and let B be the event rolling a 3 on the second roll. Are A and B independent events?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
A die is rolled twice. Let A be the event rolling a 6 on the first roll and let B be the event rolling a 3 on the second roll. Are A and B independent events?

### Response: Let's think step by step.
A and B are independent events if rolling a 6 on the first roll does not affect the probability of rolling a 3 on the second roll. 
The probability of rolling a 6 on the first roll is 1/6, and the probability of rolling a 3 on the second roll is 1/6. Since the probability of rolling a 6 does not affect the probability of rolling a 3, A and B are independent events. 
 \\ 
 Therefore, the answer is: 1

Ground Truth 
 Yes, A and B are independent events as the outcome of the first roll

## Testing Cryptographic Conceptual capabilities

### Testing the fine tuned model

In [33]:
SYSTEM_PROMPT = "You are a cryptographic expert with very strong conceptual understanding of the domain. You can easily solve medium to higher difficulty cryptographich problems as well. Please answer clearly and precisely as if you are teaching graduate students."

In [30]:
def generate_response(prompt,test_model):
    # Ignore warnings
    logging.set_verbosity(logging.CRITICAL)

    # Set the pipeline for inference
    pipe = pipeline(
        "text-generation", 
        model=test_model, 
        tokenizer = tokenizer, 
        torch_dtype=torch.bfloat16, 
        device_map= {"": 0}
    )
    # Print the results
    result = pipe(f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n ### Instruction:\n{prompt}\n\n### Response: Let's think step by step.\n",do_sample=True,
        max_new_tokens=512, 
        temperature=0.9, 
        top_k=50, 
        top_p=0.95,
        num_return_sequences=1)
    return result

In [34]:
result = generate_response(crypto_dataset[2]['question'],ft_model)
print(f"Input:\n{crypto_dataset[2]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[2]['answer']}")

Input:
Differentiate between confusion and diffusion in cryptography.

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
Differentiate between confusion and diffusion in cryptography.

### Response: Let's think step by step.
Confusion and diffusion are two fundamental principles in cryptography. 

- Confusion aims to make it difficult to trace a plaintext to a particular ciphertext.
- Diffusion aims to make it difficult to tell which bits of a ciphertext came from which bits of the plaintext.

In general, a good cipher should have both: 

- Confusion in that it makes it difficult to determine the plaintext based on the ciphertext alone, and 
- Diffusion in that it makes it difficult to determine the relationships between different bits of the ciphertext.

This ensures that a cipher is less susceptible to cryptanalytic attacks. 

There is no simple answer. The answer will depend on the specif

In [35]:
result = generate_response(crypto_dataset[8]['question'],ft_model)
print(f"Input:\n{crypto_dataset[8]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[8]['answer']}")

Input:
What are hash functions useful for? Give an example.

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What are hash functions useful for? Give an example.

### Response: Let's think step by step.
A hash function is useful for making an efficient mapping from a space of keys into a smaller space called hash table. For example, a hash function can map the set of words in English into a table of 256 entries, where the entries are labeled 0-255. 

255 is equal to $2^8 - 1$, which suggests that the table is indexed using a byte (or 8 bits). 

Furthermore, the hash function is usually represented by a program, which is run by the computer to compute the mapping. 

This mapping is also called a hash function. However, in general, a hash function may not be programmable, but simply a mathematical function. 

For example, let's consider a hash function that maps an integer key (a positive in

In [36]:
result = generate_response(crypto_dataset[10]['question'],ft_model)
print(f"Input:\n{crypto_dataset[10]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[10]['answer']}")

Input:
What is one way function in cryptography?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What is one way function in cryptography?

### Response: Let's think step by step.
One way function is a function that does not allow recovering of the original inputs. This is achieved by ensuring that the outputs are unique and the inverse of the function is not feasible to find. In cryptography, this property makes the function desirable in constructing encryption schemes, providing security for sensitive information.  
 
To achieve this, the encryption scheme relies on the property of the one way function to ensure that finding the original inputs or the plaintext message is not feasible. 

The idea of using a one way function in cryptography has been utilized by various techniques such as hashing, which maps a variable length input to a fixed length output, thereby providing a method for 

In [37]:
result = generate_response(crypto_dataset[15]['question'],ft_model)
print(f"Input:\n{crypto_dataset[15]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[15]['answer']}")

Input:
What are the drawbacks of symmetric encryption?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What are the drawbacks of symmetric encryption?

### Response: Let's think step by step.
Symmetric encryption is faster than public-key encryption, but it also has some drawbacks. One drawback is that both parties involved in the communication need to share the key, which can be difficult to manage and securely distribute. This drawback also results in another problem, which is that symmetric encryption does not have built-in authentication, making it vulnerable to man-in-the-middle attacks. However, authentication can be provided by using cryptographic hashing and a digital signature, which is often used in combination with symmetric encryption. 

Another drawback is that a single key is used to encrypt and decrypt the data, which reduces the possibility of leaking the key. However, it 

### Testing the base  model

In [38]:
result = generate_response(crypto_dataset[2]['question'],base_model)
print(f"Input:\n{crypto_dataset[2]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[2]['answer']}")

Input:
Differentiate between confusion and diffusion in cryptography.

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
Differentiate between confusion and diffusion in cryptography.

### Response: Let's think step by step.
In cryptography, confusion and diffusion are two important concepts that are used to protect the confidentiality of data.
Confusion is achieved by using a complex function to scramble the plaintext, making it difficult to determine the relationship between the plaintext and the ciphertext.
Diffusion is achieved by using a function that spreads the information in the plaintext evenly throughout the ciphertext, making it difficult to determine the original plaintext from the ciphertext.
The main difference between confusion and diffusion is that confusion is achieved through the use of a complex function, while diffusion is achieved through the use of a function that sprea

In [44]:
result = generate_response(crypto_dataset[8]['question'],base_model)
print(f"Input:\n{crypto_dataset[8]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text']}")
print(f"Ground Truth \n {crypto_dataset[8]['answer']}")

Input:
What are hash functions useful for? Give an example.

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What are hash functions useful for? Give an example.

### Response: Let's think step by step.
The answer is: 
Ground Truth 
 Hash functions are functions that map data of arbitrary size to fixed-size values. They are useful for verifying the integrity of data. An example of a hash function is SHA-256.


In [45]:
result = generate_response(crypto_dataset[15]['question'],base_model)
print(f"Input:\n{crypto_dataset[15]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[15]['answer']}")

Input:
What are the drawbacks of symmetric encryption?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What are the drawbacks of symmetric encryption?

### Response: Let's think step by step.
Symmetric encryption is a type of encryption where the same key is used for both encryption and decryption.
The drawbacks of symmetric encryption are:
1. The key must be kept secret. If the key is compromised, the encrypted data can be decrypted by anyone.
2. The key length can be a concern. If the key is too short, it may be vulnerable to brute-force attacks. If the key is too long, it may be difficult to manage.
3. The key must be of sufficient length to provide the desired level of security.
4. Symmetric encryption does not provide authentication. It can be used to encrypt data, but it cannot verify the identity of the sender or the integrity of the data.
5. Symmetric encryption does not provide f

In [47]:
result = generate_response(crypto_dataset[15]['question'],base_model)
print(f"Input:\n{crypto_dataset[15]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[15]['answer']}")

Input:
What are the drawbacks of symmetric encryption?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What are the drawbacks of symmetric encryption?

### Response: Let's think step by step.
The drawbacks of symmetric encryption are:
- It can be difficult to manage keys for large numbers of users or devices.
- It may not be suitable for applications where confidentiality is required, as the same key is used for both encryption and decryption.
- It may not be suitable for applications where the data is sensitive or critical, as the key is transmitted over the network or stored in a database.
- It may not be suitable for applications where the data is subject to regulation or compliance requirements, as the key is transmitted over the network or stored in a database.
- It may not be suitable for applications where the data is subject to legal or regulatory requirements, as the key is trans

## Cipher Math problems

In [48]:
result = generate_response(crypto_dataset[45]['question'],base_model)
print(f"Input:\n{crypto_dataset[45]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[45]['answer']}")

Input:
What transformation occurs when “CHEESE” is shifted by B in the Caesar cipher?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What transformation occurs when “CHEESE” is shifted by B in the Caesar cipher?

### Response: Let's think step by step.
In the Caesar cipher, each letter is shifted by a certain number of positions.
If we shift the letter "C" by B positions, we get the letter "F".
Similarly, if we shift the letter "H" by B positions, we get the letter "J".
And so on.
So, the transformation that occurs when "CHEESE" is shifted by B in the Caesar cipher is that each letter is shifted by B positions.

Ground Truth 
 When “CHEESE” is shifted by B in the Caesar cipher, the transformation is “EJGGUG”.


In [50]:
result = generate_response(crypto_dataset[45]['question'],ft_model)
print(f"Input:\n{crypto_dataset[45]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[45]['answer']}")

Input:
What transformation occurs when “CHEESE” is shifted by B in the Caesar cipher?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What transformation occurs when “CHEESE” is shifted by B in the Caesar cipher?

### Response: Let's think step by step.
A Caesar cipher is a shift cipher that is a form of mono-alphabetic encryption.  It is considered one of the earliest encryption techniques and is simple to implement.  This type of encryption shifts each letter in the plaintext by a certain number of positions.  This number can range from 1 to 25, depending on the size of the alphabet.  The Caesar cipher is also known as a shift cipher. The letter A is designated as the starting point for counting positions, and the letter that corresponds to this position after the shift is designated as B. The letter that corresponds to the next position after the shift is designated as C, and so on.  I

In [53]:
result = generate_response(crypto_dataset[49]['question'],ft_model)
print(f"Input:\n{crypto_dataset[49]['question']}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[49]['answer']}")

Input:
What is the process for decrypting the message 'yurgooaeod' using a rail fence with a depth of 2?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What is the process for decrypting the message 'yurgooaeod' using a rail fence with a depth of 2?

### Response: Let's think step by step.
To decrypt the message, we need to understand the structure of the rail fence. 
The rail fence has a depth of 2, meaning it has 2 rows. 
When we write the letters in the first row from left to right, we read the first letter, the fourth letter, the second letter, and the fifth letter. 
Then, we move to the second row, starting from the first letter. 
Let's look at the example: 
'yurgooaeod' 
Reading from left to right in the first row, we read y, u, r, g, o, e, o, d. 
Then, we move to the second row, starting from the first letter again. 
Reading from left to right in the second row, we read y, u, r, g